FORMULAIRE HTML

In [ ]:
<h1>Formulaire de saisie</h1>
    <form method="post">
        {% csrf_token %}
        <label for="radius_mean">Radius_mean:</label>
        <input type="text" id="radius_mean" name="radius_mean"><br>

        <label for="texture_mean">Texture_mean:</label>
        <input type="text" id="texture_mean" name="texture_mean"><br>

        <label for="smoothness_mean">Smoothness_mean:</label>
        <input type="text" id="smoothness_mean" name="smoothness_mean"><br>

        <label for="compactness_mean">Compactness_mean:</label>
        <input type="text" id="compactness_mean" name="compactness_mean"><br>

        <label for="symmetry_mean">Symmetry_mean:</label>
        <input type="text" id="symmetry_mean" name="symmetry_mean"><br>

        <label for="radius_se">Radius_se:</label>
        <input type="text" id="radius_se" name="radius_se"><br>

        <label for="area_se">Area_se:</label>
        <input type="text" id="area_se" name="area_se"><br>

        <label for="compactness_se">Compactness_se:</label>
        <input type="text" id="compactness_se" name="compactness_se"><br>

        <label for="concavity_se">Concavity_se:</label>
        <input type="text" id="concavity_se" name="concavity_se"><br>

        <label for="concave points_se">Concave_points_se:</label>
        <input type="text" id="concave points_se" name="concave points_se"><br>

        <label for="symmetry_se">Symmetry_se:</label>
        <input type="text" id="symmetry_se" name="symmetry_se"><br>

        <label for="fractal_dimension_se">Fractal_dimension_se:</label>
        <input type="text" id="fractal_dimension_se" name="fractal_dimension_se"><br>

        <label for="smoothness_worst">Smoothness_worst:</label>
        <input type="text" id="smoothness_worst" name="smoothness_worst"><br>

        <label for="compactness_worst">Compactness_worst:</label>
        <input type="text" id="compactness_worst" name="compactness_worst"><br>

        <label for="concave points_worst">Concave_points_worst:</label>
        <input type="text" id="concave points_worst" name="concave points_worst"><br>

        <label for="symmetry_worst">Symmetry_worst:</label>
        <input type="text" id="symmetry_worst" name="symmetry_worst"><br>

        <label for="fractal_dimension_worst">Fractal_dimension_worst:</label>
        <input type="text" id="fractal_dimension_worst" name="fractal_dimension_worst"><br>

        <input type="submit" value="Soumettre">
    </form>

Step 2 : Pipeline

In [ ]:
import pandas as pd
import joblib
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df = pd.read_csv("/content/df_cancer_clean.csv")


# On prend que les variables qu'on a décidé de garder:
var_quanti = list(df[['radius_mean', 'texture_mean','smoothness_mean', 'compactness_mean',
                                    'symmetry_mean','radius_se','area_se','compactness_se', 'concavity_se',
                                     'concave points_se', 'symmetry_se','fractal_dimension_se','smoothness_worst',
                                    'compactness_worst','concave points_worst','symmetry_worst', 'fractal_dimension_worst']].columns)

for colonne in var_quanti:
  ecart_interQ =  df[colonne].quantile(0.75) - df[colonne].quantile(0.25)
  condition = (df[colonne]<df[colonne].quantile(0.25) - ecart_interQ*1.5) | (df[colonne]>df[colonne].quantile(0.75) + ecart_interQ*1.5)
  df.loc[(condition == True), colonne] = df[colonne].median()

df.drop(['Unnamed: 0','id'], axis=1, inplace = True)

X = df[var_quanti]
y = df['diagnosis']

# Diviser les données en ensembles d'entraînement et de test avant l'oversampling :
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

# Appliquer SMOTE sur l'ensemble d'entraînement
# Utilisation de sampling_strategy='auto' pour ajustement automatique du ratio en fonction de la distribution des classes dans l'ensemble d'entraînement
smote = SMOTE(sampling_strategy='auto', random_state=32)
X_train_res, y_train_res = smote.fit_resample(X, y)

pipeline = Pipeline([
            ('Scaler', StandardScaler()),
            ('Model', LogisticRegression(random_state=32, C=1, solver='liblinear'))])

pipeline.fit(X_train_res, y_train_res)

# Enregistrement du modèle
joblib.dump(pipeline, "logistic_regression_model.pkl")

['logistic_regression_model.pkl']

step 3 : Formulaire

In [ ]:
def formulaire(request):
    if request.method == 'POST':
        radius_mean = request.POST.get('radius_mean')
        texture_mean = request.POST.get('texture_mean')
        smoothness_mean = request.POST.get('smoothness_mean')
        compactness_mean = request.POST.get('compactness_mean')
        symmetry_mean = request.POST.get('symmetry_mean')
        radius_se = request.POST.get('radius_se')
        area_se = request.POST.get('area_se')
        compactness_se = request.POST.get('compactness_se')
        concavity_se = request.POST.get('concavity_se')
        concave_points_se = request.POST.get('concave points_se')
        symmetry_se = request.POST.get('symmetry_se')
        fractal_dimension_se = request.POST.get('fractal_dimension_se')
        smoothness_worst = request.POST.get('smoothness_worst')
        compactness_worst = request.POST.get('compactness_worst')
        concave_points_worst = request.POST.get('concave points_worst')
        symmetry_worst = request.POST.get('symmetry_worst')
        fractal_dimension_worst = request.POST.get('fractal_dimension_worst')

        # Entrees du site
        data = {
            'radius_mean': [radius_mean],
            'texture_mean': [texture_mean],
            'smoothness_mean': [smoothness_mean],
            'compactness_mean': [compactness_mean],
            'symmetry_mean': [symmetry_mean],
            'radius_se': [radius_se],
            'area_se': [area_se],
            'compactness_se': [compactness_se],
            'concavity_se': [concavity_se],
            'concave points_se': [concave_points_se],
            'symmetry_se': [symmetry_se],
            'fractal_dimension_se': [fractal_dimension_se],
            'smoothness_worst': [smoothness_worst],
            'compactness_worst': [compactness_worst],
            'concave points_worst': [concave_points_worst],
            'symmetry_worst': [symmetry_worst],
            'fractal_dimension_worst': [fractal_dimension_worst],
        }

        df_donnee_site = pd.DataFrame(data)

        # Supression colonnes inutilisées dans modèle
        # df_donnee_site.drop("xx", axis = 1, inplace=True)

        # Import pipeline pré-entrainée
        pipeline_loaded = joblib.load('C:/Users/vika_/Documents/1_Formation_Vika/1_Wild_Code_School/Projet_3/logistic_regression_model.pkl')

        # Prediction
        prediction = pipeline_loaded.predict(df_donnee_site)[0]

        message = f"Résultat de la prediction : {prediction}"
        return HttpResponse(message)

    return render(request, 'form.html')